In [ ]:
!pip3 install langchain-community pypdf

In [ ]:
from google.colab import userdata
LANG_SMITH_API_KEY = userdata.get('LANGSMITH_API_KEY')

In [ ]:
LANGSMITH_TRACING = "true"

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
file_path = '/content/nke-10k-2023.pdf'
loader = PyPDFLoader(file_path)
docs = loader.load()

In [ ]:
print(len(docs))

In [ ]:
print(f'{docs[0].page_content[:200]}')

In [ ]:
print(docs[0].metadata)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

In [ ]:
len(all_splits)

In [ ]:
!pip uninstall -y google-ai-generativelanguage
!pip install -qU google-ai-generativelanguage==0.6.15 google-generativeai==0.8.5 langchain-google-genai

In [ ]:
GEMINI_API_KEY = userdata.get('geminiapikey')

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model='models/embedding-001',
    google_api_key=GEMINI_API_KEY
)

vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

In [ ]:
!pip install -qU langchain-core

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [ ]:
ids = vector_store.add_documents(documents=all_splits)

In [ ]:
results = vector_store.similarity_search(
    "How many distribution centers does Nike have in the US?"
)

print(results[0])

In [ ]:
results = vector_store.similarity_search_with_score("What was Nike's revenue in 2023?")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)